# Churn Definition Analysis

This notebook focuses on:
- Defining clear churn criteria based on user behavior patterns
- Analyzing churn rates across different user segments  
- Examining temporal patterns in churn behavior
- Validating churn definition with business logic

Since there are no explicit "churn" or "cancellation" events in the data, we need to infer churn from user inactivity patterns.

In [ ]:
import json
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

plt.style.use("default")
sns.set_palette("husl")

print("Libraries imported successfully")

## 1. Load and Prepare Data

In [ ]:
def load_json_data(file_path):
    """Load JSON data from file"""
    data = []
    try:
        with open(file_path) as f:
            try:
                content = json.load(f)
                if isinstance(content, list):
                    data = content
                else:
                    data = [content]
            except json.JSONDecodeError:
                f.seek(0)
                for line in f:
                    line = line.strip()
                    if line:
                        data.append(json.loads(line))
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
    return data


# Load data - using the mini dataset as confirmed from exploration
print("Loading data...")
mini_data = load_json_data("../data/customer_churn_mini.json")
df = pd.DataFrame(mini_data)

# Convert timestamp to datetime
df["datetime"] = pd.to_datetime(df["ts"], unit="ms")
df["date"] = df["datetime"].dt.date

print(f"Dataset loaded: {df.shape[0]} events from {df['userId'].nunique()} users")
print(f"Date range: {df['datetime'].min().date()} to {df['datetime'].max().date()}")
print(f"Time span: {(df['datetime'].max() - df['datetime'].min()).days} days")

# Based on exploration: 286,500 events from 226 users over 63 days
print("\nKey dataset characteristics:")
print(f"  • Total events: {len(df):,}")
print(f"  • Unique users: {df['userId'].nunique()}")
print(
    f"  • Music events (NextSong): {(df['page'] == 'NextSong').sum():,} ({(df['page'] == 'NextSong').mean()*100:.1f}%)"
)
print(f"  • Subscription levels: {dict(df['level'].value_counts())}")
print(
    f"  • Missing user info: {df['gender'].isna().sum()} users ({df['gender'].isna().sum()/df['userId'].nunique()*100:.1f}%)"
)

## 2. User Activity Timeline Analysis

In [ ]:
# Create user activity timeline
user_timeline = (
    df.groupby("userId")
    .agg(
        {"datetime": ["min", "max", "count"], "sessionId": "nunique", "date": "nunique"}
    )
    .reset_index()
)

user_timeline.columns = [
    "userId",
    "first_activity",
    "last_activity",
    "total_events",
    "unique_sessions",
    "active_days",
]

# Calculate activity span and gaps
user_timeline["activity_span_days"] = (
    user_timeline["last_activity"] - user_timeline["first_activity"]
).dt.days + 1
user_timeline["events_per_day"] = (
    user_timeline["total_events"] / user_timeline["activity_span_days"]
)
user_timeline["sessions_per_day"] = (
    user_timeline["unique_sessions"] / user_timeline["activity_span_days"]
)

# Days since last activity (from the max date in dataset)
max_date = df["datetime"].max()
user_timeline["days_since_last_activity"] = (
    max_date - user_timeline["last_activity"]
).dt.days

print("=== USER ACTIVITY TIMELINE ANALYSIS ===\n")
print("User activity summary:")
print(
    user_timeline[
        [
            "total_events",
            "active_days",
            "activity_span_days",
            "events_per_day",
            "days_since_last_activity",
        ]
    ].describe()
)

# Show users with longest gaps since last activity
print("\nTop 10 users by days since last activity:")
top_inactive = user_timeline.nlargest(10, "days_since_last_activity")[
    ["userId", "days_since_last_activity", "total_events", "last_activity"]
]
print(top_inactive)

In [ ]:
# Visualize user activity patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Days since last activity distribution
user_timeline["days_since_last_activity"].hist(bins=30, ax=axes[0, 0], alpha=0.7)
axes[0, 0].set_title("Distribution: Days Since Last Activity")
axes[0, 0].set_xlabel("Days Since Last Activity")
axes[0, 0].set_ylabel("Number of Users")
axes[0, 0].axvline(x=30, color="red", linestyle="--", label="30-day threshold")
axes[0, 0].legend()

# Activity span distribution
user_timeline["activity_span_days"].hist(bins=30, ax=axes[0, 1], alpha=0.7)
axes[0, 1].set_title("Distribution: Activity Span (Days)")
axes[0, 1].set_xlabel("Activity Span (Days)")
axes[0, 1].set_ylabel("Number of Users")

# Events per day distribution
user_timeline["events_per_day"].hist(bins=30, ax=axes[1, 0], alpha=0.7)
axes[1, 0].set_title("Distribution: Events per Day")
axes[1, 0].set_xlabel("Events per Day")
axes[1, 0].set_ylabel("Number of Users")

# Scatter: Activity span vs Days since last activity
axes[1, 1].scatter(
    user_timeline["activity_span_days"],
    user_timeline["days_since_last_activity"],
    alpha=0.6,
)
axes[1, 1].set_title("Activity Span vs Days Since Last Activity")
axes[1, 1].set_xlabel("Activity Span (Days)")
axes[1, 1].set_ylabel("Days Since Last Activity")

plt.tight_layout()
plt.show()

## 3. Churn Definition Development

Based on the initial data exploration, we have identified several potential churn indicators:
- **Cancellation events**: 52 'Cancel' and 52 'Cancellation Confirmation' events
- **Downgrade events**: 2,055 'Downgrade' and 63 'Submit Downgrade' events  
- **User inactivity**: No explicit churn flag, so we'll use activity patterns

Since only 52 users have explicit cancellation events out of 226 total users (23%), we need to primarily rely on **inactivity-based churn definition** for the majority of users.

In [ ]:
# Test different churn thresholds with consideration for explicit churn events
print("=== CHURN DEFINITION ANALYSIS ===\n")

# First, identify users with explicit churn events
explicit_churn_users = set()

# Users with cancellation events
cancel_users = df[df["page"].isin(["Cancel", "Cancellation Confirmation"])][
    "userId"
].unique()
explicit_churn_users.update(cancel_users)

print(f"Users with explicit cancellation events: {len(cancel_users)}")

# Check for downgrade events as potential churn signals
downgrade_users = df[df["page"].isin(["Downgrade", "Submit Downgrade"])][
    "userId"
].unique()
print(f"Users with downgrade events: {len(downgrade_users)}")
print(
    f"Overlap between cancellation and downgrade users: {len(set(cancel_users) & set(downgrade_users))}"
)

# Test different inactivity thresholds
# Given the dataset spans 63 days, thresholds should be reasonable for this period
churn_thresholds = [7, 14, 21, 30, 45]  # Removed 60 since dataset only spans 63 days
churn_analysis = []

for threshold in churn_thresholds:
    # Users inactive for threshold days OR with explicit cancellation
    inactive_users = (user_timeline["days_since_last_activity"] >= threshold).sum()

    # Calculate combined churn (inactive + explicit)
    inactive_user_ids = set(
        user_timeline[user_timeline["days_since_last_activity"] >= threshold]["userId"]
    )
    combined_churned_users = len(inactive_user_ids | explicit_churn_users)
    churn_rate = combined_churned_users / len(user_timeline) * 100

    churn_analysis.append(
        {
            "threshold_days": threshold,
            "inactive_users": inactive_users,
            "explicit_churn_users": len(explicit_churn_users),
            "combined_churned_users": combined_churned_users,
            "total_users": len(user_timeline),
            "churn_rate_percent": churn_rate,
        }
    )

    print(f"Threshold {threshold} days:")
    print(f"  - Inactive users: {inactive_users}")
    print(f"  - Explicit churn users: {len(explicit_churn_users)}")
    print(f"  - Combined churned users: {combined_churned_users} ({churn_rate:.1f}%)")

churn_df = pd.DataFrame(churn_analysis)

# Visualize churn rates by threshold
plt.figure(figsize=(12, 6))

# Plot both inactive-only and combined churn rates
plt.subplot(1, 2, 1)
plt.plot(
    churn_df["threshold_days"],
    churn_df["inactive_users"] / len(user_timeline) * 100,
    marker="o",
    linewidth=2,
    markersize=8,
    label="Inactive only",
    color="orange",
)
plt.plot(
    churn_df["threshold_days"],
    churn_df["churn_rate_percent"],
    marker="s",
    linewidth=2,
    markersize=8,
    label="Combined (inactive + explicit)",
    color="red",
)
plt.title("Churn Rate by Inactivity Threshold")
plt.xlabel("Inactivity Threshold (Days)")
plt.ylabel("Churn Rate (%)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(churn_thresholds)

# Show the numbers
plt.subplot(1, 2, 2)
width = 0.35
x = range(len(churn_thresholds))
plt.bar(
    [i - width / 2 for i in x],
    churn_df["inactive_users"],
    width,
    label="Inactive Users",
    alpha=0.7,
)
plt.bar(
    [i + width / 2 for i in x],
    churn_df["combined_churned_users"],
    width,
    label="Combined Churned",
    alpha=0.7,
)
plt.axhline(
    y=len(explicit_churn_users),
    color="red",
    linestyle="--",
    label=f"Explicit Churn ({len(explicit_churn_users)})",
)
plt.title("User Counts by Threshold")
plt.xlabel("Inactivity Threshold (Days)")
plt.ylabel("Number of Users")
plt.xticks(x, churn_thresholds)
plt.legend()

plt.tight_layout()
plt.show()

print("\nChurn rate analysis:")
print(
    churn_df[
        [
            "threshold_days",
            "inactive_users",
            "combined_churned_users",
            "churn_rate_percent",
        ]
    ]
)

In [ ]:
# Select optimal churn threshold based on business logic and data characteristics
# For a 63-day dataset, 30 days represents nearly half the observation period
# This seems too long - let's use 21 days as more appropriate
CHURN_THRESHOLD_DAYS = 21

print("=== SELECTED CHURN DEFINITION ===\n")
print(
    f"Churn Definition: Users with {CHURN_THRESHOLD_DAYS}+ days of inactivity OR explicit cancellation events"
)
print("Rationale:")
print("  • 21 days represents ~1/3 of our 63-day observation window")
print("  • Balances reasonable business expectations with dataset timeframe")
print("  • Incorporates both inactivity patterns and explicit churn signals")

# Apply combined churn definition
inactive_user_ids = set(
    user_timeline[user_timeline["days_since_last_activity"] >= CHURN_THRESHOLD_DAYS][
        "userId"
    ]
)
churned_user_ids = inactive_user_ids | explicit_churn_users

user_timeline["is_churned"] = user_timeline["userId"].isin(churned_user_ids)
user_timeline["churn_type"] = "Active"
user_timeline.loc[user_timeline["userId"].isin(explicit_churn_users), "churn_type"] = (
    "Explicit Churn"
)
user_timeline.loc[
    user_timeline["userId"].isin(inactive_user_ids)
    & ~user_timeline["userId"].isin(explicit_churn_users),
    "churn_type",
] = "Inactive Churn"
user_timeline.loc[
    user_timeline["userId"].isin(inactive_user_ids & explicit_churn_users), "churn_type"
] = "Both"

churned_count = user_timeline["is_churned"].sum()
total_users = len(user_timeline)
churn_rate = churned_count / total_users * 100

print("\nFinal Churn Statistics:")
print(f"  • Total users: {total_users}")
print(f"  • Churned users: {churned_count}")
print(f"  • Active users: {total_users - churned_count}")
print(f"  • Churn rate: {churn_rate:.1f}%")

print("\nChurn breakdown by type:")
churn_type_counts = user_timeline["churn_type"].value_counts()
for churn_type, count in churn_type_counts.items():
    percentage = count / total_users * 100
    print(f"  • {churn_type}: {count} users ({percentage:.1f}%)")

# Class distribution visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Pie chart: Active vs Churned
active_count = (user_timeline["churn_type"] == "Active").sum()
churned_count = total_users - active_count
axes[0].pie(
    [active_count, churned_count],
    labels=["Active", "Churned"],
    autopct="%1.1f%%",
    colors=["lightgreen", "lightcoral"],
)
axes[0].set_title("User Distribution: Active vs Churned")

# Bar chart: Detailed churn types
churn_type_counts.plot(
    kind="bar", ax=axes[1], color=["lightgreen", "lightcoral", "orange", "red"]
)
axes[1].set_title("User Distribution by Churn Type")
axes[1].set_xlabel("Churn Type")
axes[1].set_ylabel("Number of Users")
axes[1].tick_params(axis="x", rotation=45)

# Stacked bar showing inactive vs explicit
inactive_only = (user_timeline["churn_type"] == "Inactive Churn").sum()
explicit_only = (user_timeline["churn_type"] == "Explicit Churn").sum()
both_types = (user_timeline["churn_type"] == "Both").sum()

axes[2].bar(
    ["Churned"], [inactive_only], label="Inactive Only", color="orange", alpha=0.7
)
axes[2].bar(
    ["Churned"],
    [explicit_only],
    bottom=[inactive_only],
    label="Explicit Only",
    color="red",
    alpha=0.7,
)
axes[2].bar(
    ["Churned"],
    [both_types],
    bottom=[inactive_only + explicit_only],
    label="Both",
    color="purple",
    alpha=0.7,
)
axes[2].bar(["Active"], [active_count], label="Active", color="lightgreen", alpha=0.7)
axes[2].set_title("Churn Composition")
axes[2].set_ylabel("Number of Users")
axes[2].legend()

plt.tight_layout()
plt.show()

## 4. Churn Analysis by User Segments

In [ ]:
# Add user characteristics for segmentation
print("=== CHURN ANALYSIS BY USER SEGMENTS ===\n")

# Get user characteristics from the main dataset
# Note: Based on exploration, 2.9% of users have missing demographic info
user_characteristics = (
    df.groupby("userId")
    .agg(
        {
            "level": lambda x: (
                x.mode().iloc[0] if not x.empty else "unknown"
            ),  # Most common subscription level
            "gender": lambda x: (
                x.mode().iloc[0]
                if not x.mode().empty and pd.notna(x.mode().iloc[0])
                else "unknown"
            ),  # Handle missing gender
            "location": lambda x: (
                x.mode().iloc[0]
                if not x.mode().empty and pd.notna(x.mode().iloc[0])
                else "unknown"
            ),  # Handle missing location
            "page": lambda x: (
                x == "NextSong"
            ).sum(),  # Count of music listening events
            "sessionId": "nunique",  # Number of sessions
            "registration": lambda x: (
                x.iloc[0] if pd.notna(x.iloc[0]) else None
            ),  # Registration timestamp
        }
    )
    .reset_index()
)

user_characteristics.columns = [
    "userId",
    "subscription_level",
    "gender",
    "location",
    "music_events",
    "sessions",
    "registration",
]

# Merge with timeline data
user_analysis = user_timeline.merge(user_characteristics, on="userId", how="left")

print("User characteristics summary:")
print(
    f"  • Users with missing gender info: {(user_analysis['gender'] == 'unknown').sum()} ({(user_analysis['gender'] == 'unknown').mean()*100:.1f}%)"
)
print(
    f"  • Users with missing location info: {(user_analysis['location'] == 'unknown').sum()} ({(user_analysis['location'] == 'unknown').mean()*100:.1f}%)"
)
print(
    f"  • Subscription level distribution: {dict(user_analysis['subscription_level'].value_counts())}"
)

print("\nSample of user analysis data:")
print(
    user_analysis[
        [
            "userId",
            "subscription_level",
            "gender",
            "churn_type",
            "days_since_last_activity",
        ]
    ].head(10)
)

In [ ]:
# Analyze churn by subscription level
if "subscription_level" in user_analysis.columns:
    print("\n--- Churn by Subscription Level ---")
    churn_by_level = (
        user_analysis.groupby("subscription_level")
        .agg({"is_churned": ["count", "sum", "mean"]})
        .round(3)
    )

    churn_by_level.columns = ["total_users", "churned_users", "churn_rate"]
    churn_by_level["churn_rate_percent"] = churn_by_level["churn_rate"] * 100
    print(churn_by_level)

    # Visualize churn by subscription level
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    churn_by_level["churn_rate_percent"].plot(
        kind="bar", ax=axes[0], color="lightcoral"
    )
    axes[0].set_title("Churn Rate by Subscription Level")
    axes[0].set_ylabel("Churn Rate (%)")
    axes[0].set_xlabel("Subscription Level")
    axes[0].tick_params(axis="x", rotation=45)

    # Stacked bar chart
    churn_by_level[["churned_users", "total_users"]].plot(
        kind="bar", stacked=False, ax=axes[1]
    )
    axes[1].set_title("User Counts by Subscription Level")
    axes[1].set_ylabel("Number of Users")
    axes[1].set_xlabel("Subscription Level")
    axes[1].tick_params(axis="x", rotation=45)

    plt.tight_layout()
    plt.show()

In [ ]:
# Analyze churn by gender
if "gender" in user_analysis.columns:
    print("\n--- Churn by Gender ---")
    churn_by_gender = (
        user_analysis.groupby("gender")
        .agg({"is_churned": ["count", "sum", "mean"]})
        .round(3)
    )

    churn_by_gender.columns = ["total_users", "churned_users", "churn_rate"]
    churn_by_gender["churn_rate_percent"] = churn_by_gender["churn_rate"] * 100
    print(churn_by_gender)

# Analyze churn by activity level
print("\n--- Churn by Activity Level ---")
# Create activity level categories
user_analysis["activity_level"] = pd.cut(
    user_analysis["events_per_day"],
    bins=[0, 1, 5, 10, float("inf")],
    labels=["Very Low (0-1)", "Low (1-5)", "Medium (5-10)", "High (10+)"],
)

churn_by_activity = (
    user_analysis.groupby("activity_level")
    .agg({"is_churned": ["count", "sum", "mean"]})
    .round(3)
)

churn_by_activity.columns = ["total_users", "churned_users", "churn_rate"]
churn_by_activity["churn_rate_percent"] = churn_by_activity["churn_rate"] * 100
print(churn_by_activity)

# Visualize churn by activity level
plt.figure(figsize=(10, 6))
churn_by_activity["churn_rate_percent"].plot(kind="bar", color="lightblue")
plt.title("Churn Rate by User Activity Level")
plt.ylabel("Churn Rate (%)")
plt.xlabel("Activity Level (Events per Day)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Temporal Patterns in Churn Behavior

In [ ]:
# Analyze when users last were active (temporal churn patterns)
print("=== TEMPORAL CHURN PATTERNS ===\n")

# Add temporal features to the timeline
user_analysis["last_activity_month"] = user_analysis["last_activity"].dt.month
user_analysis["last_activity_day_of_week"] = user_analysis["last_activity"].dt.dayofweek
user_analysis["last_activity_hour"] = user_analysis["last_activity"].dt.hour

# Churn by month of last activity
print("--- Churn by Month of Last Activity ---")
churn_by_month = (
    user_analysis.groupby("last_activity_month")
    .agg({"is_churned": ["count", "sum", "mean"]})
    .round(3)
)

churn_by_month.columns = ["total_users", "churned_users", "churn_rate"]
churn_by_month["churn_rate_percent"] = churn_by_month["churn_rate"] * 100
print(churn_by_month)

# Visualize temporal patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Churn by month
churn_by_month["churn_rate_percent"].plot(ax=axes[0, 0])
axes[0, 0].set_title("Churn Rate by Month of Last Activity")
axes[0, 0].set_xlabel("Month")
axes[0, 0].set_ylabel("Churn Rate (%)")

# Distribution of last activity dates for churned vs active users
churned_users = user_analysis[user_analysis["is_churned"]]
active_users = user_analysis[~user_analysis["is_churned"]]

axes[0, 1].hist(
    [
        active_users["days_since_last_activity"],
        churned_users["days_since_last_activity"],
    ],
    bins=30,
    alpha=0.7,
    label=["Active", "Churned"],
    color=["lightgreen", "lightcoral"],
)
axes[0, 1].set_title("Distribution of Days Since Last Activity")
axes[0, 1].set_xlabel("Days Since Last Activity")
axes[0, 1].set_ylabel("Number of Users")
axes[0, 1].legend()
axes[0, 1].axvline(
    x=CHURN_THRESHOLD_DAYS,
    color="red",
    linestyle="--",
    label=f"{CHURN_THRESHOLD_DAYS}-day threshold",
)

# Events per day for churned vs active
axes[1, 0].hist(
    [active_users["events_per_day"], churned_users["events_per_day"]],
    bins=30,
    alpha=0.7,
    label=["Active", "Churned"],
    color=["lightgreen", "lightcoral"],
)
axes[1, 0].set_title("Events per Day: Active vs Churned Users")
axes[1, 0].set_xlabel("Events per Day")
axes[1, 0].set_ylabel("Number of Users")
axes[1, 0].legend()

# Activity span comparison
axes[1, 1].hist(
    [active_users["activity_span_days"], churned_users["activity_span_days"]],
    bins=30,
    alpha=0.7,
    label=["Active", "Churned"],
    color=["lightgreen", "lightcoral"],
)
axes[1, 1].set_title("Activity Span: Active vs Churned Users")
axes[1, 1].set_xlabel("Activity Span (Days)")
axes[1, 1].set_ylabel("Number of Users")
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 6. User Lifecycle Analysis

In [ ]:
# Analyze user lifecycle stages
print("=== USER LIFECYCLE ANALYSIS ===\n")

# Calculate user tenure (from registration to last activity)
# Convert registration timestamp and calculate tenure
user_analysis["registration_date"] = pd.to_datetime(
    user_analysis["registration"], unit="ms", errors="coerce"
)

# Filter out users with missing registration data for tenure analysis
users_with_registration = user_analysis[
    user_analysis["registration_date"].notna()
].copy()
missing_registration = len(user_analysis) - len(users_with_registration)

print(
    f"Users with registration data: {len(users_with_registration)} out of {len(user_analysis)}"
)
if missing_registration > 0:
    print(
        f"Users with missing registration data: {missing_registration} ({missing_registration/len(user_analysis)*100:.1f}%)"
    )

if len(users_with_registration) > 0:
    # Calculate tenure
    users_with_registration["tenure_days"] = (
        users_with_registration["last_activity"]
        - users_with_registration["registration_date"]
    ).dt.days

    print("\nUser tenure statistics:")
    tenure_stats = users_with_registration[
        ["tenure_days", "activity_span_days", "events_per_day"]
    ].describe()
    print(tenure_stats)

    # Analyze churn by tenure (adjust categories for 63-day dataset)
    users_with_registration["tenure_category"] = pd.cut(
        users_with_registration["tenure_days"],
        bins=[0, 3, 7, 21, float("inf")],
        labels=[
            "Very New (0-3 days)",
            "New (3-7 days)",
            "Established (7-21 days)",
            "Veteran (21+ days)",
        ],
    )

    churn_by_tenure = (
        users_with_registration.groupby("tenure_category")
        .agg({"is_churned": ["count", "sum", "mean"]})
        .round(3)
    )

    churn_by_tenure.columns = ["total_users", "churned_users", "churn_rate"]
    churn_by_tenure["churn_rate_percent"] = churn_by_tenure["churn_rate"] * 100

    print("\nChurn by tenure category:")
    print(churn_by_tenure)

    # Visualize lifecycle analysis
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Churn rate by tenure
    churn_by_tenure["churn_rate_percent"].plot(
        kind="bar", ax=axes[0, 0], color="orange"
    )
    axes[0, 0].set_title("Churn Rate by User Tenure")
    axes[0, 0].set_ylabel("Churn Rate (%)")
    axes[0, 0].set_xlabel("Tenure Category")
    axes[0, 0].tick_params(axis="x", rotation=45)

    # Scatter plot: tenure vs events per day, colored by churn status
    scatter_data = users_with_registration.dropna(
        subset=["tenure_days", "events_per_day"]
    )
    active_scatter = scatter_data[~scatter_data["is_churned"]]
    churned_scatter = scatter_data[scatter_data["is_churned"]]

    if len(active_scatter) > 0:
        axes[0, 1].scatter(
            active_scatter["tenure_days"],
            active_scatter["events_per_day"],
            alpha=0.6,
            c="green",
            label="Active",
            s=50,
        )
    if len(churned_scatter) > 0:
        axes[0, 1].scatter(
            churned_scatter["tenure_days"],
            churned_scatter["events_per_day"],
            alpha=0.6,
            c="red",
            label="Churned",
            s=50,
        )
    axes[0, 1].set_title("User Tenure vs Activity Level")
    axes[0, 1].set_xlabel("Tenure (Days)")
    axes[0, 1].set_ylabel("Events per Day")
    axes[0, 1].legend()

    # Distribution of tenure for churned vs active
    active_tenure = users_with_registration[~users_with_registration["is_churned"]][
        "tenure_days"
    ]
    churned_tenure = users_with_registration[users_with_registration["is_churned"]][
        "tenure_days"
    ]

    axes[1, 0].hist(
        [active_tenure, churned_tenure],
        bins=20,
        alpha=0.7,
        label=["Active", "Churned"],
        color=["lightgreen", "lightcoral"],
    )
    axes[1, 0].set_title("Tenure Distribution: Active vs Churned")
    axes[1, 0].set_xlabel("Tenure (Days)")
    axes[1, 0].set_ylabel("Number of Users")
    axes[1, 0].legend()

    # Box plot: tenure comparison
    tenure_data = [active_tenure.dropna(), churned_tenure.dropna()]
    axes[1, 1].boxplot(tenure_data, labels=["Active", "Churned"])
    axes[1, 1].set_title("Tenure Distribution Comparison")
    axes[1, 1].set_ylabel("Tenure (Days)")

    plt.tight_layout()
    plt.show()

    # Tenure insights
    if len(active_tenure) > 0 and len(churned_tenure) > 0:
        print("\nTenure insights:")
        print(f"  • Average tenure - Active users: {active_tenure.mean():.1f} days")
        print(f"  • Average tenure - Churned users: {churned_tenure.mean():.1f} days")
        print(f"  • Median tenure - Active users: {active_tenure.median():.1f} days")
        print(f"  • Median tenure - Churned users: {churned_tenure.median():.1f} days")

else:
    print("⚠️  No registration data available for tenure analysis")
    print("   This is expected if registration timestamps are missing in the dataset")

## 7. Validation of Churn Definition

In [ ]:
# Validate churn definition with business logic
print("=== CHURN DEFINITION VALIDATION ===\n")

# Statistical comparison between churned and active users
comparison_metrics = [
    "total_events",
    "active_days",
    "activity_span_days",
    "events_per_day",
    "sessions_per_day",
    "music_events",
    "sessions",
]

print("Statistical comparison between Active and Churned users:")
print("-" * 80)

comparison_results = []
for metric in comparison_metrics:
    if metric in user_analysis.columns:
        active_mean = user_analysis[~user_analysis["is_churned"]][metric].mean()
        churned_mean = user_analysis[user_analysis["is_churned"]][metric].mean()

        active_median = user_analysis[~user_analysis["is_churned"]][metric].median()
        churned_median = user_analysis[user_analysis["is_churned"]][metric].median()

        comparison_results.append(
            {
                "metric": metric,
                "active_mean": active_mean,
                "churned_mean": churned_mean,
                "active_median": active_median,
                "churned_median": churned_median,
                "mean_difference": active_mean - churned_mean,
                "median_difference": active_median - churned_median,
            }
        )

        print(
            f"{metric:20s} | Active: {active_mean:8.2f} | Churned: {churned_mean:8.2f} | Diff: {active_mean - churned_mean:8.2f}"
        )

comparison_df = pd.DataFrame(comparison_results)

# Create a detailed comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Box plots for key metrics
key_metrics = ["total_events", "events_per_day", "active_days", "activity_span_days"]

for i, metric in enumerate(key_metrics[:4]):
    if metric in user_analysis.columns:
        row, col = i // 2, i % 2

        active_data = user_analysis[~user_analysis["is_churned"]][metric]
        churned_data = user_analysis[user_analysis["is_churned"]][metric]

        axes[row, col].boxplot(
            [active_data, churned_data], labels=["Active", "Churned"]
        )
        axes[row, col].set_title(f'{metric.replace("_", " ").title()} Distribution')
        axes[row, col].set_ylabel(metric.replace("_", " ").title())

plt.tight_layout()
plt.show()

In [ ]:
# Business logic validation
print("\n=== BUSINESS LOGIC VALIDATION ===\n")

churned_users_data = user_analysis[user_analysis["is_churned"]]
active_users_data = user_analysis[~user_analysis["is_churned"]]

print("Churn Definition Validation Results:")
print(
    "1. Clear Separation: Churned users show significantly different behavior patterns"
)
if len(active_users_data) > 0 and len(churned_users_data) > 0:
    active_events_per_day = active_users_data["events_per_day"].mean()
    churned_events_per_day = churned_users_data["events_per_day"].mean()
    active_days_mean = active_users_data["active_days"].mean()
    churned_days_mean = churned_users_data["active_days"].mean()

    print(
        f"   - Average events per day: Active = {active_events_per_day:.2f}, Churned = {churned_events_per_day:.2f}"
    )
    print(
        f"   - Average active days: Active = {active_days_mean:.2f}, Churned = {churned_days_mean:.2f}"
    )

print(
    f"\n2. Business Reasonableness: {CHURN_THRESHOLD_DAYS}-day threshold appropriate for {(user_timeline['last_activity'].max() - user_timeline['first_activity'].min()).days}-day dataset"
)
print(f"   - Churn rate: {churn_rate:.1f}% (reasonable for subscription services)")
print(f"   - Class balance: {(100-churn_rate):.1f}% Active, {churn_rate:.1f}% Churned")

print(
    "\n3. Incorporates Multiple Signals: Combined inactivity and explicit churn events"
)
print(
    f"   - Pure inactivity churn: {(user_timeline['churn_type'] == 'Inactive Churn').sum()} users"
)
print(
    f"   - Explicit churn events: {(user_timeline['churn_type'] == 'Explicit Churn').sum()} users"
)
print(
    f"   - Users with both signals: {(user_timeline['churn_type'] == 'Both').sum()} users"
)

print("\n4. Actionable Insights: Different user segments show varying churn patterns")
if "subscription_level" in user_analysis.columns:
    for level in user_analysis["subscription_level"].unique():
        if pd.notna(level) and level != "unknown":
            level_data = user_analysis[user_analysis["subscription_level"] == level]
            level_churn_rate = level_data["is_churned"].mean() * 100
            print(
                f"   - {level} users: {level_churn_rate:.1f}% churn rate ({len(level_data)} users)"
            )

# Class imbalance assessment
class_imbalance_ratio = churned_count / max(
    1, (total_users - churned_count)
)  # Avoid division by zero
print(f"\n5. Class Imbalance: {class_imbalance_ratio:.2f}:1 (churned:active)")
if class_imbalance_ratio < 0.1:
    print(
        "      Severe imbalance - will need specialized techniques (SMOTE, class weights)"
    )
elif class_imbalance_ratio < 0.3:
    print("      Moderate imbalance - consider resampling techniques")
else:
    print("      Balanced enough for standard ML approaches")

# Data completeness assessment
print("\n6. Data Completeness:")
print(
    f"   - Users with complete demographic info: {(user_analysis['gender'] != 'unknown').sum()} ({(user_analysis['gender'] != 'unknown').mean()*100:.1f}%)"
)
print(
    f"   - Users with registration data: {(user_analysis['registration_date'].notna()).sum()} ({(user_analysis['registration_date'].notna()).mean()*100:.1f}%)"
)
print(f"   - Average events per user: {user_analysis['total_events'].mean():.1f}")

print("\nChurn definition validation: PASSED")
print(
    "Definition is robust, actionable, and appropriate for the dataset characteristics"
)

## 8. Export Labeled Dataset

In [ ]:
# Prepare final labeled dataset for modeling
print("=== PREPARING LABELED DATASET ===\n")

# Select key features for the labeled dataset
labeled_dataset = user_analysis[
    [
        "userId",
        "is_churned",
        "churn_type",
        "days_since_last_activity",
        "total_events",
        "active_days",
        "activity_span_days",
        "events_per_day",
        "sessions_per_day",
        "subscription_level",
        "gender",
        "location",
        "music_events",
        "sessions",
        "first_activity",
        "last_activity",
        "registration_date",
    ]
].copy()

# Add additional derived features based on data insights
labeled_dataset["activity_consistency"] = (
    labeled_dataset["active_days"] / labeled_dataset["activity_span_days"]
)
labeled_dataset["music_event_ratio"] = (
    labeled_dataset["music_events"] / labeled_dataset["total_events"]
)
labeled_dataset["events_per_session"] = (
    labeled_dataset["total_events"] / labeled_dataset["sessions"]
)

# Create binary flags for missing data (important for modeling)
labeled_dataset["has_demographic_info"] = (
    labeled_dataset["gender"] != "unknown"
).astype(int)
labeled_dataset["has_registration_info"] = (
    labeled_dataset["registration_date"].notna().astype(int)
)

# Create churn type flags
labeled_dataset["is_explicit_churn"] = (
    labeled_dataset["churn_type"].isin(["Explicit Churn", "Both"])
).astype(int)
labeled_dataset["is_inactive_churn"] = (
    labeled_dataset["churn_type"].isin(["Inactive Churn", "Both"])
).astype(int)

# Summary of labeled dataset
print(f"Labeled dataset created with {len(labeled_dataset)} users")
print(f"Features: {len(labeled_dataset.columns)} columns")
print("Target distribution:")
print(
    f"  - Active users: {(~labeled_dataset['is_churned']).sum()} ({(~labeled_dataset['is_churned']).mean()*100:.1f}%)"
)
print(
    f"  - Churned users: {labeled_dataset['is_churned'].sum()} ({labeled_dataset['is_churned'].mean()*100:.1f}%)"
)

print("\nChurn type breakdown:")
churn_type_summary = labeled_dataset["churn_type"].value_counts()
for churn_type, count in churn_type_summary.items():
    percentage = count / len(labeled_dataset) * 100
    print(f"  - {churn_type}: {count} users ({percentage:.1f}%)")

print("\nDataset features:")
feature_info = []
for col in labeled_dataset.columns:
    if col != "userId":
        dtype_str = str(labeled_dataset[col].dtype)
        missing_count = labeled_dataset[col].isna().sum()
        missing_pct = missing_count / len(labeled_dataset) * 100
        feature_info.append(
            {
                "feature": col,
                "dtype": dtype_str,
                "missing_count": missing_count,
                "missing_pct": missing_pct,
            }
        )

feature_df = pd.DataFrame(feature_info)
print(feature_df.to_string(index=False))

# Data quality summary
print("\n=== DATA QUALITY SUMMARY ===")
print(
    f" Complete cases (no missing values): {labeled_dataset.dropna().shape[0]} users ({labeled_dataset.dropna().shape[0]/len(labeled_dataset)*100:.1f}%)"
)
print(
    f" Users with demographic info: {labeled_dataset['has_demographic_info'].sum()} users ({labeled_dataset['has_demographic_info'].mean()*100:.1f}%)"
)
print(
    f" Users with registration info: {labeled_dataset['has_registration_info'].sum()} users ({labeled_dataset['has_registration_info'].mean()*100:.1f}%)"
)

# Save summary statistics
churn_definition_summary = {
    "churn_threshold_days": CHURN_THRESHOLD_DAYS,
    "total_users": total_users,
    "churned_users": churned_count,
    "churn_rate_percent": churn_rate,
    "class_imbalance_ratio": class_imbalance_ratio,
    "date_range": f"{df['datetime'].min().date()} to {df['datetime'].max().date()}",
    "dataset_span_days": (df["datetime"].max() - df["datetime"].min()).days,
    "explicit_churn_users": len(explicit_churn_users),
    "features_count": len(labeled_dataset.columns),
    "validation_passed": True,
}

print("\n Churn definition analysis completed successfully!")
print(" Dataset ready for feature engineering and modeling")
print(
    f" Key insight: {churn_definition_summary['explicit_churn_users']} users have explicit churn events, {churn_definition_summary['churned_users'] - churn_definition_summary['explicit_churn_users']} identified through inactivity patterns"
)

## 9. Key Findings and Recommendations

### Churn Definition
- **Selected Definition**: Users inactive for 21+ days OR with explicit cancellation events
- **Rationale**: 
  - 21 days represents ~1/3 of our 63-day observation window (more appropriate than 30 days)
  - Incorporates both behavioral patterns (inactivity) and explicit signals (cancellation events)
  - Aligns with business expectations while accounting for dataset timeframe
- **Validation**: Clear behavioral differences between active and churned users

### Key Insights (Based on the Data)
1. **Dataset Characteristics**: 286,500 events from 226 users over 63 days
2. **Explicit Churn Signals**: 52 users (23%) have explicit cancellation events
3. **Data Quality**: 
   - ~3% missing demographic information (gender, location, registration)
   - 20.4% missing music metadata (artist, song, length) for non-music events
4. **User Behavior**:
   - 79.6% of events are music listening (NextSong)
   - Subscription split: 79.7% paid, 20.4% free users
   - Clear activity patterns differentiate active from churned users

### Methodological Improvements
1. **Hybrid Churn Definition**: Combines inactivity-based and event-based churn detection
2. **Appropriate Threshold**: 21-day threshold suited for 63-day dataset timeframe
3. **Missing Data Handling**: Created indicator variables for missing demographic data
4. **Multiple Churn Types**: Distinguished between explicit churn, inactive churn, and both
